## 메모리 chain으로 연결하기.

ConversationSummaryBufferMemory
- save, load를 하지 않고 chain으로 연결해 자동으로 즉시 메모리 활용하기.
- 요약하고 최근 대화 내용을 기억하지만, 챗봇으로 만들 경우 아래와 같이 template안에 이전 요약 내용을 넣어줘야 기억할 수 있음.
- 요약내용은 verbose 덕에 확인 가능하며 System 다음 문장들임.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

In [ ]:

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,

    # memory는 template 안 대화 기록 저장하는 공간 찾음.
    memory_key="chat_history"
)

# 템플릿 안에 history 저장하기 위해 chat_history 작성.
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

# 문제점 1
'''
- memory에는 저장되지만, prompt에서는 저장되지 않음.   
  -> question값만 전달하고 있어서 과거 내용을 기억 못함.

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True   # 값을 chain에 보내고 실행했을 떄 chain의 프롬프트 로그 확인 가능
)
'''

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True   # 값을 chain에 보내고 실행했을 떄 chain의 프롬프트 로그 확인 가능
)

In [8]:
chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [2]:
chain.predict(question="I line in Seoul")

"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [10]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

In [11]:
memory.load_memory_variables({})

{'history': 'System: The human asks the AI for their name, but the AI responds that it does not have access to personal information like names. The human shares their name, "Nice," and the AI greets Nice and offers assistance. When the human, Nico, shares their name, the AI reiterates its lack of access to personal information.\nHuman: What is my name?\nAI: I\'m sorry, I do not know your name as I am an AI assistant and do not have access to personal information.\nHuman: What is my name?\nAI: I\'m sorry, I do not have access to personal information such as your name.'}